# K-En Yakın Komşu(KNN)

- Tahminler gözlem benzerliğine göre yapılır.

- Bana arkadaşını söyle, sana kim olduğunu söyleyeyim.

-----

# KNN Basamakları

- Komşu sayısını belirle. (K)

- Bilinmeyen nokta ile diğer tüm noktalar arasındaki uzaklıkları hesapla.

- Uzaklıkları sırala ve belirlenen 'k' sayısına göre en yakın olan 'k' gözlemi seç.

- Sınıflandırmada en sık sınıf değeri olarak, regresyonda ise ortalama değeri tahmin değeri olarak ver.

--------

# 1. Model (KNN)

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

from warnings import filterwarnings
filterwarnings('ignore')

In [5]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()

In [6]:
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A


In [10]:
dms = pd.get_dummies(df[["League", "Division", "NewLeague"]])

In [11]:
dms.head()

,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
1,0,1,0,1,0,1
2,1,0,0,1,1,0
3,0,1,1,0,0,1
4,0,1,1,0,0,1
5,1,0,0,1,1,0


In [12]:
X_ = df.drop(["Salary", "League", "Division", "NewLeague"], axis = 1).astype("float64")

In [13]:
X_.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0


In [14]:
X = pd.concat([X_, dms[["League_N", "Division_W", "NewLeague_N"]]], axis = 1)

In [15]:
X.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0,1,1,1
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0,0,1,0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0,1,0,1
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0,1,0,1
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0,0,1,0


In [16]:
y = df["Salary"]

In [17]:
y.head()

1    475.0
2    480.0
3    500.0
4     91.5
5    750.0
Name: Salary, dtype: float64

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [19]:
knn_model = KNeighborsRegressor().fit(X_train, y_train)

In [20]:
knn_model

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [21]:
k = knn_model.n_neighbors  # komşu sayısı
k

5

------

# 2. Tahmin (KNN)

In [22]:
knn_model.predict(X_test)

array([ 510.3334,  808.3334,  772.5   ,  125.5   , 1005.    ,  325.5   ,
        216.5   ,  101.5   ,  982.    ,  886.6666,  590.    ,  901.6666,
        831.6666,  157.5   ,  393.    , 1005.    ,  735.5   ,   97.    ,
        884.4   ,  302.    ,  450.    ,  817.6666,  832.6666,  392.3334,
        528.    ,   81.6   ,  735.    ,  470.    ,  722.5   ,  101.    ,
         90.5   ,   74.6   ,  748.3334,  217.    ,  280.5334, 1044.5   ,
        955.    ,  232.    ,   78.6   ,  529.    ,   77.6   ,  106.5   ,
        516.6666,  593.6666, 1005.    ,  649.1666,  715.    ,  101.5   ,
        134.5   ,  810.    ,  743.    ,  521.3334,  664.3334,  195.    ,
        102.4   ,  728.5   ,  488.    ,  962.5   ,  230.8334, 1040.    ,
        885.    ,  542.    ,  720.4   ,  571.    ,  735.    ,   81.6   ])

In [23]:
y_pred_test = knn_model.predict(X_test)

In [24]:
np.sqrt(mean_squared_error(y_test, y_pred_test))

426.6570764525201

In [26]:
RMSE = []
for k in range(1, 11):
    knn_model = KNeighborsRegressor(n_neighbors = k)
    knn_model.fit(X_train, y_train)
    y_pred_train = knn_model.predict(X_train)
    rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    RMSE.append(rmse)
    print("k = {} için RMSE değeri : {}".format(k, rmse))

k = 1 için RMSE değeri : 0.0
k = 2 için RMSE değeri : 179.52761335480352
k = 3 için RMSE değeri : 205.20157172291863
k = 4 için RMSE değeri : 220.5139794876305
k = 5 için RMSE değeri : 239.64671325413764
k = 6 için RMSE değeri : 243.5904190007242
k = 7 için RMSE değeri : 258.1478781634636
k = 8 için RMSE değeri : 266.05374203349805
k = 9 için RMSE değeri : 269.73782093553376
k = 10 için RMSE değeri : 271.2798300436963


-------

# 3. Model Doğrulama / Model Tuning (KNN)

In [27]:
np.arange(1, 30, 1)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [28]:
knn_params = {"n_neighbors" : np.arange(1, 30, 1)}

In [29]:
knn_params

{'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])}

In [30]:
knn = KNeighborsRegressor()

In [31]:
knn_cv_model = GridSearchCV(knn, knn_params, cv = 10)

In [32]:
knn_cv_model.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [33]:
knn_cv_model.best_params_["n_neighbors"]

8

In [35]:
# CV değerleri her zaman bizim için daha değerlidir!

In [36]:
RMSE = []
RMSE_CV = []

for k in range(1, 11):
    knn_model = KNeighborsRegressor(n_neighbors = k)
    knn_model.fit(X_train, y_train)
    
    y_pred_train = knn_model.predict(X_train)
    
    rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    cross_validation = cross_val_score(knn_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error").mean()
    rmse_cv = np.sqrt(-1 * cross_validation)
    
    RMSE.append(rmse)
    RMSE_CV.append(rmse_cv)
    
    print("k = {} için RMSE değeri : {}, RMSE_CV değeri : {}".format(k, rmse, rmse_cv))

k = 1 için RMSE değeri : 0.0, RMSE_CV değeri : 325.39475147063825
k = 2 için RMSE değeri : 179.52761335480352, RMSE_CV değeri : 293.24000183333817
k = 3 için RMSE değeri : 205.20157172291863, RMSE_CV değeri : 283.7486667487823
k = 4 için RMSE değeri : 220.5139794876305, RMSE_CV değeri : 286.3240222024089
k = 5 için RMSE değeri : 239.64671325413764, RMSE_CV değeri : 290.0705466132226
k = 6 için RMSE değeri : 243.5904190007242, RMSE_CV değeri : 298.1263115575851
k = 7 için RMSE değeri : 258.1478781634636, RMSE_CV değeri : 294.77070479194987
k = 8 için RMSE değeri : 266.05374203349805, RMSE_CV değeri : 291.98672028891235
k = 9 için RMSE değeri : 269.73782093553376, RMSE_CV değeri : 295.7162739573105
k = 10 için RMSE değeri : 271.2798300436963, RMSE_CV değeri : 301.31047022701154


In [48]:
RMSE = []
RMSE_CV = []

for k in range(1, 11):
    knn_model = KNeighborsRegressor(n_neighbors = k)
    knn_model.fit(X_train, y_train)
    
    y_pred_test = knn_model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    cross_validation = cross_val_score(knn_model, X_train, y_train, cv = 10, scoring = "neg_mean_squared_error").mean()
    rmse_cv = np.sqrt(-1 * cross_validation)
    
    RMSE.append(rmse)
    RMSE_CV.append(rmse_cv)
    
    print("k = {} için RMSE değeri : {}, RMSE_CV değeri : {}".format(k, rmse, rmse_cv))

k = 1 için RMSE değeri : 455.03925390751965, RMSE_CV değeri : 325.39475147063825
k = 2 için RMSE değeri : 415.99629571490965, RMSE_CV değeri : 293.24000183333817
k = 3 için RMSE değeri : 420.6765370082348, RMSE_CV değeri : 283.7486667487823
k = 4 için RMSE değeri : 428.8564674588792, RMSE_CV değeri : 286.3240222024089
k = 5 için RMSE değeri : 426.6570764525201, RMSE_CV değeri : 290.0705466132226
k = 6 için RMSE değeri : 423.5071669008732, RMSE_CV değeri : 298.1263115575851
k = 7 için RMSE değeri : 414.9361222421057, RMSE_CV değeri : 294.77070479194987
k = 8 için RMSE değeri : 413.7094731463598, RMSE_CV değeri : 291.98672028891235
k = 9 için RMSE değeri : 417.84419990871265, RMSE_CV değeri : 295.7162739573105
k = 10 için RMSE değeri : 421.6252180741266, RMSE_CV değeri : 301.31047022701154


In [44]:
knn_tuned = KNeighborsRegressor(n_neighbors = knn_cv_model.best_params_["n_neighbors"])

In [45]:
knn_tuned.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=8, p=2,
                    weights='uniform')

In [46]:
y_pred_test = knn_tuned.predict(X_test)

In [47]:
np.sqrt(mean_squared_error(y_test, y_pred_test))

413.7094731463598

-------------